In [41]:
import pandas as pd
import random
import holidays
from progiter import ProgIter
import json
import datetime
import os

def generate_backtest(name, year, instruments, sample_size, start, end, start_time, end_time, pvars, sg_arrays):
    depth_files = os.listdir(depth_path)
    idict = {
        instr: {d.split('.')[1]: d for d in depth_files if instr in d}
        for instr in instruments
    }
    main = pd.DataFrame({"day": pd.date_range(start=start, end=end, freq='D')})
    # remove weekends
    main = main[~main.day.dt.day_name().isin(['Sunday', 'Saturday'])]
    # remove bank holidays in major countries
    to_remove_holidays = (
        list(holidays.UnitedKingdom(years=[year]).keys()) +
        list(holidays.UnitedStates(years=[year]).keys()) +
        list(holidays.NewYorkStockExchange(years=[year]).keys()) +
        list(holidays.EuropeanCentralBank(years=[year]).keys())
    )
    main = main[~main.day.isin(pd.to_datetime(to_remove_holidays))]
    date_strings = main.day.apply(lambda x: str(x).split(' ')[0]).tolist()
    if sample_size > 0:
        date_strings = sorted(random.sample(date_strings, k=sample_size))
    print(f'n days -> {len(date_strings)}')

    contracts = []
    for instrument in instruments:
        for date_string in date_strings:
            if date_string in idict[instrument]:
                contracts.append(idict[instrument][date_string].split('.')[0])
    
    contracts = set(contracts)
    print(f'contracts -> {contracts}')

    print("copy depth files..")
    depth_files_copied = []
    for instrument in instruments:
        print(f'current instrument -> {instrument}')
        for date_string in ProgIter(date_strings):
            if not date_string in idict[instrument].keys():
                print(f'date string missing from files, skip -> {date_string}')
                continue
                
            depth_file = idict[instrument][date_string]
            if not os.path.exists(f'{base_depth_path}/{depth_file}'):
                os.system(f'cp {depth_path}/{depth_file} {base_depth_path}/{depth_file}')
                depth_files_copied.append(f"{base_depth_path}/{depth_file}")

    date_ranges = []
    for instrument in instruments:
        for date_string in date_strings:
            if not date_string in idict[instrument].keys():
                continue
                
            current = {
                "data_file": f"{idict[instrument][date_string].split('.')[0]}.scid",
                "start": f"{date_string} {start_time}",
                "end": f"{date_string} {end_time}"
            }
            date_ranges.append(current)

    res = {
        "name": name,
        "date_ranges": date_ranges,
        "sg_arrays": sg_arrays,
        "pvars": pvars
    }
    with open(f"{bt_base_path}/{res['name']}.json", 'w') as f:
        json.dump(res, f, indent=4, sort_keys=False)
    print('done.')
    return depth_files_copied



### Make backtest

In [44]:
name = "some_bt"
year = 2024
sample_size = 0
instruments = ["MES"]
start, end = '2024-10-01', '2024-10-05'
start_time, end_time = "14:00:00", "15:00:00"
pvars = [
    {"name": "account_value", "type": "vector", "study_name": "Account Balance Graph", "pkey": 2},
]
sg_arrays = []


bt_base_path = "/mnt/c/trading/backtest"
base_data_path = "/mnt/c/SierraChart/Data"
base_depth_path = f"{base_data_path}/MarketDepthData"
history_path = "/mnt/h/trading/history"
year_path = f"{history_path}/{year}"
data_path = f"{year_path}/Data"
depth_path = f"{data_path}/MarketDepthData"

depth_files_copied = generate_backtest(name, year, instruments, sample_size, start, end, start_time, end_time, pvars, sg_arrays)

n days -> 4
contracts -> {'MES-202412-CME-USD'}
copy depth files..
current instrument -> MES
 100.00% 4/4... rate=0.47 Hz, eta=0:00:00, total=0:00:08
done.


### Remove .depth files

In [40]:
os.system(f'rm -rf {base_depth_path}/*.depth')

0